# Szczecin 2015-2024

Analizujemy dane dotyczące podatku od nieruchomości w Szczecinie w latach 2015-2024 płaconego za powierzchnię mieszkalną.

In [1]:
from lib import szczecin_2015_2024

# Podział na osoby fizyczne i prawne

In [2]:
def add_index_border(styler):
    df = styler.data
    index = df.index
    index_df = index.to_frame()
    styler = styler.set_table_styles(
        {
            tuple(t): [{'selector': '', 'props': 'border-top: 1px solid;'}]
            for t in index_df.reset_index(drop=True).groupby(index_df.columns[0]).first().iloc[1:].itertuples()
        },
        overwrite=False,
        axis=1,
    )
    return styler

(
    szczecin_2015_2024.get_summary_df()
    .set_index(['taxpayer_type', 'year'])
    .style
    .format('{:,.0f}')
    .pipe(add_index_border)
    .bar()
)

Płatnicy będący osobami prawnymi stanowią niewielką część płatników i kont.
W przypadku osób prawnych liczba płatników i kont jest identyczna.
Ta liczba rosła w danym okresie od 296 do 522, ale powierzchnia mieszkalna, za którą płaciły osoby prawne nie zmieniała się istotnie.

Płatnicy będący osobami fizycznymi stanowią zdecydowaną większość płatników i kont - rozróżnienie staje się istotne zapewne ze względu na małżeństwa ze wspólnotą majątkową.
Powierzchnia mieszkalna, za którą płaciły osoby fizyczne oraz liczba płatników rosły przez cały okres.

Wzrost powierzchni wiąże się z budową mieszkań, a wzrost liczby płatników - ze wzrostem liczby osób posiadających mieszkanie.

# Dane szczegółowe dotyczące osób fizycznych

Dane nt. osób fizycznych mamy też w rozbiciu na grupy podatników.
Grupy definiowane są przez powierzchnię mieszkalną, od której podatnicy płacili podatek:

- do 50m2
- od 50m2 do 100m2
- od 100m2 do 200m2
- od 200m2 do 500m2
- od 500m2 do 2000m2
- powyżej 2000m2

Dla uproszczenia zredukujmy podział na dwie grupy, poniżej i powyżej 200m2,
co można uznać za punkt odcięcia między wysokim komfortem mieszkalnym w mieście, a inwestycją w nieruchomości lub luksusem.

In [3]:
cutoff = 200

two_way_split_details_df = (
    szczecin_2015_2024.get_details_df()
    .assign(group=lambda df: df.apply(lambda row: f'below {cutoff}' if row['area_max'] <= cutoff else f'above {cutoff}', axis=1))
    .groupby(['group', 'year'])
    [['n_taxpayers', 'n_accounts', 'total_area']]
    .sum()
)
(
    two_way_split_details_df
    .style
    .format('{:,.0f}')
    .pipe(add_index_border)
    .bar()
)

Mając uproszczony podział na te dwie grupy, sprawdźmy jak przyrastała powierzchnia mieszkalna rok do roku.

Dla każdego roku zobaczymy jak wzrost powierzchni lokali mieszkalnej był rozłożony między grupy.

In [4]:
(
    two_way_split_details_df
    .reset_index()
    .pivot(index='year', columns='group', values='total_area')
    .diff(1).dropna()
    .apply(lambda row: row / row.sum(), axis=1)
    .style
    .format('{:.2%}')
    .bar()
)

group,above 200,below 200
year,,
2016,6.58%,93.42%
2017,7.18%,92.82%
2018,5.99%,94.01%
2019,10.49%,89.51%
2020,11.59%,88.41%
2021,10.67%,89.33%
2022,12.64%,87.36%
2023,17.33%,82.67%
2024,17.59%,82.41%


- W latach 2015-2018 grupie "powyżej 200m2" przypadało 6-7% przyrostu powierzchni mieszkalnej
- W latach 2019-2022 było to już 10-12%
- Od roku 2023 jest to 17%